<a href="https://colab.research.google.com/github/MosesIbe23/CSE450_Machine_Learning/blob/main/notebooks/starter_publishing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
restart = True
epoch_to_pickup = 0

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
path = ''

## Functions for downloading text


## III. Build the model

Next, we'll build our model. Up until this point, you've been using the Keras symbolic, or imperative API for creating your models. Doing something like:

    model = tf.keras.models.Sequentla()
    model.add(tf.keras.layers.Dense(80, activation='relu))
    etc...

However, tensorflow has another way to build models called the Functional API, which gives us a lot more control over what happens inside the model. You can read more about [the differences and when to use each here](https://blog.tensorflow.org/2019/01/what-are-symbolic-and-imperative-apis.html).

We'll use the functional API for our RNN in this example. This will involve defining our model as a custom subclass of `tf.keras.Model`.

If you're not familiar with classes in python, you might want to review [this quick tutorial](https://www.w3schools.com/python/python_classes.asp), as well as [this one on class inheritance](https://www.w3schools.com/python/python_inheritance.asp).

Using a functional model is important for our situation because we're not just training it to predict a single character for a single sequence, but as we make predictions with it, we need it to remember those predictions as use that memory as it makes new predictions.


In [ ]:
# sherlock_text = getMyText()

File 'austen.txt' found locally. Using it.


In [13]:
from tensorflow.keras.layers import StringLookup
import numpy as np
import os
import time
import random
import contextlib
import io
import re
import string
import gc  # Import the garbage collector module
import os
import re
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import TextVectorization

In [14]:
from google.colab import drive
drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)
path = "/content/drive/My Drive/The Project Gutenberg eBook of Mexican a.txt"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mounted at /content/drive


In [15]:
def preprocess_text(text):

    text = text.replace("Project Gutenberg", "")
    text = text.replace("Gutenberg", "")

    # Remove carriage returns
    text = text.replace("\r", "")

    # fix quotes
    text = text.replace("“", "\"")
    text = text.replace("”", "\"")

    # Replace any capital letter at the start of a word with ^ followed by the lowercase letter
    text = re.sub(r"(?<![a-zA-Z])([A-Z])", lambda match: f"^{match.group(0).lower()}", text)

    # Replace all other capital letters with lowercase
    text = re.sub(r"([A-Z])", lambda match: f"{match.group(0).lower()}", text)

    # Remove duplicate whitespace
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"\n+", "\n", text)
    text = re.sub(r"\t+", "\t", text)

    # Replace whitespace characters with special words
    text = re.sub(r"(\t)", r" zztabzz ", text)
    text = re.sub(r"(\n)", r" zznewlinezz ", text)
    text = re.sub(r"(\s)", r" zzspacezz ", text)

    # Split before and after punctuation
    for punctuation in string.punctuation:
        text = text.replace(punctuation, f" {punctuation} ")

    return text

def postprocess_text(text):

    # Replace special words with whitespace characters
    text = text.replace("zztabzz", "\t")
    text = text.replace("zznewlinezz", "\n")
    text = text.replace("zzspacezz", " ")

    # Remake capital letters at beginning of words
    text = re.sub(r"\^([a-z])", lambda match: f"{match.group(1).upper()}", text)

    text = text.replace("^", "")

    return text

with open(path, 'r', encoding='utf-8') as file:
    text = file.read()

def Mytext():
    file_name = 'Mytext.txt'
    file_url = 'https://www.gutenberg.org/cache/epub/74836/pg74836.txt'
    local_dir = 'my_files'  # Directory to save the file
    local_path = os.path.join(local_dir, file_name)

    try:
        # Ensure the directory exists
        if not os.path.exists(local_dir):
            os.makedirs(local_dir)

        # Check if the file exists locally
        if os.path.exists(local_path):
            print(f"File '{file_name}' found locally. Using it.")
        else:
            print(f"File '{file_name}' not found locally. Downloading it.")
            # Download the file
            downloaded_path = tf.keras.utils.get_file(file_name, file_url)

            # Save the downloaded file to the designated local directory
            with open(downloaded_path, 'rb') as source_file:
                with open(local_path, 'wb') as dest_file:
                    dest_file.write(source_file.read())

        # Read the file's contents
        with open(local_path, 'rb') as file:
            text = file.read().decode(encoding='utf-8')

        return preprocess_text(text)

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

Mytext()

File 'Mytext.txt' not found locally. Downloading it.
718314/718314 ━━━━━━━━━━━━━━━━━━━━ 1s 1us/step


'\ufeff ^ the zzspacezz ebook zzspacezz of zzspacezz  ^ mexican zzspacezz archaeology zzspacezz  ^ this zzspacezz ebook zzspacezz is zzspacezz for zzspacezz the zzspacezz use zzspacezz of zzspacezz anyone zzspacezz anywhere zzspacezz in zzspacezz the zzspacezz  ^ united zzspacezz  ^ states zzspacezz and zzspacezz most zzspacezz other zzspacezz parts zzspacezz of zzspacezz the zzspacezz world zzspacezz at zzspacezz no zzspacezz cost zzspacezz and zzspacezz with zzspacezz almost zzspacezz no zzspacezz restrictions zzspacezz whatsoever .  zzspacezz  ^ you zzspacezz may zzspacezz copy zzspacezz it ,  zzspacezz give zzspacezz it zzspacezz away zzspacezz or zzspacezz re - use zzspacezz it zzspacezz under zzspacezz the zzspacezz terms zzspacezz of zzspacezz the zzspacezz  ^ license zzspacezz included zzspacezz with zzspacezz this zzspacezz ebook zzspacezz or zzspacezz online zzspacezz at zzspacezz www . gutenberg . org .  zzspacezz  ^ if zzspacezz you zzspacezz are zzspacezz not zzspacezz loc

In [16]:

vocab_text = Mytext()

vocab_size = 8192
sequence_length = 128

vectorize_layer = TextVectorization(
      standardize='lower',
      split='whitespace',
      max_tokens=vocab_size,
      output_mode='int',
      #output_sequence_length=sequence_length
      )

vectorize_layer.adapt([vocab_text])

vocabulary = vectorize_layer.get_vocabulary()


with open(path + "vocabulary.txt", "w") as file:
  for word in vocabulary:
      file.write(word + "\n")

with open(path + "vocabulary.txt", "r") as file:
    vocabulary = [word.strip() for word in file.readlines()]
    vocabulary = vocabulary

vectorize_layer = TextVectorization(
    vocabulary=vocabulary,
    standardize='lower',
    split='whitespace',
    max_tokens=vocab_size,
    output_mode='int',
    #output_sequence_length=sequence_length
      )

print(vocabulary[:20])
print(vocabulary[-20:])


File 'Mytext.txt' found locally. Using it.
['', '[UNK]', 'zzspacezz', '^', ',', 'the', 'of', '.', 'and', 'in', 'a', 'to', '-', 'was', 'with', 'is', '_', 'as', 'were', 'that']
['elective', 'eldest', 'elab', 'ekel', 'ekbalam', 'ek', 'ein', 'egg', 'efficient', 'effective', 'edition', 'edible', 'edging', 'ecstasy', 'economy', 'economical', 'eaters', 'eastwards', 'ease', 'earth’s']


In [36]:

# This function will generate our sequence pairs:
def split_input_target(sequence):
    input_ids = sequence[:-1]
    target_ids = sequence[1:]
    return input_ids, target_ids

# This function will create the dataset
def text_to_dataset(text):
  all_ids = vectorize_layer(text)
  ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
  del all_ids
  sequences = ids_dataset.batch(sequence_length+1, drop_remainder=True)
  del ids_dataset

  # Call the function for every sequence in our list to create a new dataset
  # of input->target pairs
  dataset = sequences.map(split_input_target)
  del sequences

  # shuffle

  return dataset

vocab_ds = text_to_dataset(vocab_text)

def text_from_ids(ids):
  text = ''.join([vocabulary[index] for index in ids])
  return postprocess_text(text)

vocabulary_adjusted = vocabulary
vocabulary_adjusted[0] = '[UNK]'
vocabulary_adjusted[1] = ''

words_from_ids = tf.keras.layers.StringLookup(vocabulary=vocabulary_adjusted, invert=True)

for input_example, target_example in vocab_ds.take(1):
    print("Input: ")
    print(input_example)
    print(text_from_ids(input_example))
    print(words_from_ids(input_example))
    print("Target: ")
    print(target_example)
    print(text_from_ids(target_example))

BATCH_SIZE = 64
BUFFER_SIZE = 10000

def setup_dataset(dataset):
  dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))
  return dataset

# Rebatch the dataset
vocab_ds = setup_dataset(vocab_ds)


Input: 
tf.Tensor(
[5827    3    5    2 1264    2    6    2    3   49    2 2952    2    3
   30    2 1264    2   15    2   33    2    5    2  200    2    6    2
 2257    2 5719    2    9    2    5    2    3  822    2    3  252    2
    8    2   64    2   57    2 1583    2    6    2    5    2  322    2
   22    2   77    2 4270    2    8    2   14    2  293    2   77    2
 3170    2 4504    7    2    3  168    2   60    2 1369    2   26    4
    2  380    2   26    2  852    2   36    2 4859   12  200    2   26
    2  144    2    5    2  471    2    6    2    5    2    3  888    2
  792    2   14    2   30    2 1264    2   36    2 2743    2   22    2
 1522    7], shape=(128,), dtype=int64)
﻿The ebook of Mexican archaeology This ebook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the License included with this ebook or online at w

In [57]:
# Use the vectorized dataset `vocab_ds` from your preprocessed data
# Vocabulary size, sequence length, and embedding dimension

# Parameters
vocab_size = len(vocabulary)  # Vocabulary size
sequence_length = 5  # Input sequence length
embedding_dim = 256  # Dimensionality of the embedding layer
lstm_units = 512  # Number of units in each LSTM layer
batch_size = 64  # Batch size for training
epochs = 150  # Number of epochs for training

# Define the model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=sequence_length),
    LSTM(units=lstm_units, return_sequences=True),
    LSTM(units=lstm_units, return_sequences=True),  # Ensure return_sequences=True
    Dense(units=vocab_size, activation='softmax')  # Predict next token probabilities
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',  # One-hot encoded target loss
    metrics=['accuracy']
)

# View the model summary
print("Model Summary:")
model.build(input_shape=(None, sequence_length))
model.summary()

# Convert the target to one-hot encoded format
def preprocess_dataset(dataset):
    return dataset.map(lambda x, y: (x, tf.one_hot(y, depth=vocab_size)))

# Preprocess dataset for one-hot encoded targets
vocab_ds_one_hot = preprocess_dataset(vocab_ds)

# Train the model
print("Training the model...")
model.fit(vocab_ds_one_hot, epochs=epochs, verbose=1)

# Save the trained model
model_save_path = "trained_text_generation_model.h5"
model.save(model_save_path)
print(f"Model saved at: {model_save_path}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model Summary:


Model: "sequential_25"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_26 (Embedding)             │ (None, 5, 256)              │       2,097,152 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_58 (LSTM)                       │ (None, 5, 512)              │       1,574,912 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_59 (LSTM)                       │ (None, 5, 512)              │       2,099,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_25 (Dense)                     │ (None, 5, 8192)             │       4,202,496 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,973,760 (38.05 MB)

 Trainable params: 9,973,760 (38.05 MB)

 Non-trainable params: 0 (0.00 B)

Training the model...
Epoch 1/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 11s 239ms/step - accuracy: 0.3755 - loss: 6.2339
Epoch 2/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 9s 265ms/step - accuracy: 0.4281 - loss: 4.2153
Epoch 3/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 8s 235ms/step - accuracy: 0.4296 - loss: 4.1864
Epoch 4/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 11s 247ms/step - accuracy: 0.4284 - loss: 4.1683
Epoch 5/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 257ms/step - accuracy: 0.4265 - loss: 4.0612
Epoch 6/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 9s 221ms/step - accuracy: 0.4295 - loss: 3.8445
Epoch 7/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 11s 238ms/step - accuracy: 0.4339 - loss: 3.7327
Epoch 8/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 11s 250ms/step - accuracy: 0.4735 - loss: 3.6799
Epoch 9/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 263ms/step - accuracy: 0.4740 - loss: 3.6462
Epoch 10/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 9s 230ms/step - accuracy: 0.4735 - loss: 3.5877
Epoch 11/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 224ms/step - accuracy: 0.4738 - loss: 3.5387
Epoch 12/150

Model saved at: trained_text_generation_model.h5


In [58]:
#Save the model
model_save_path = "/path/to/save/text_generation_model.h5"  # Replace with your path
model.save(model_save_path)
print(f"Model saved at: {model_save_path}")

def generate_text(model, tokenizer, start_string, num_generate=500):
    # Use the TextVectorization layer directly for conversion
    input_eval = tokenizer([start_string]).numpy()[0]
    input_eval = tf.expand_dims(input_eval, 0)  # Add batch dimension
    generated_text = []

    for _ in range(num_generate):
        predictions = model(input_eval)
        predicted_id = tf.random.categorical(predictions[0], num_samples=1)[-1, 0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)  # Update input with predicted token
        generated_text.append(vocabulary[predicted_id])

    final_text = " ".join(generated_text).replace(" zzspacezz ", " ")
    return postprocess_text(final_text)

# Generate sample text
start_string = "The world seemed like such a peaceful place"
# Pass vectorize_layer as the tokenizer
generated_text = generate_text(model, vectorize_layer, start_string)
print(f"Generated Text:\n{generated_text}")


Model saved at: /path/to/save/text_generation_model.h5
Generated Text:
voyages effort 267 weave calculations beast tiripemé voyages 234 skirt dimensions handle fauna implication amantlan overlay xxiii unnecessarily policy equanimity identify unlucky lend gained wines disposal wielded solstice strained fertility repetition sacrificios benque splendid danger columns retirement hidden priestly heart alliance world wears wrappings funerary stripes enob slate chac tending miquiztli slaves hide proving portend earthquake legally brazier human formidable fig moulded oath tepoztecatl injury xaltocan eclipse 214 execution no criminal long simply solicit mayapan assist translucent sound purpose journeys sotuta probable 344 garden solution slavery chert fishing masterpieces motive splits stellar mannikin lowest baked inalienable astronomy wet complex partisan 337 eldest infant 7 tended midnight eastward steal tower cakes expend frieze secured religious tale slip mammal sprinkling stripes underwen